In [1]:
!nvidia-smi

Tue Jan 21 14:13:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:07:00.0 Off |                    0 |
| N/A   28C    P0             59W /  400W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
!pip uninstall transformers --yes #You'll need a special, different transformers
!pip install -q -U datasets peft trl bitsandbytes transformers[torch] scipy tqdm scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
from huggingface_hub import interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Path to the checkpoint
checkpoint_path = "results/checkpoint-9500"

# Load the model
model = AutoModelForCausalLM.from_pretrained(checkpoint_path)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

# Optional: Move the model to GPU
model = model.to("cuda")


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [5]:
def generate_response(prompt, model, tokenizer, max_length=50, temperature=0.7, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_length=max_length + len(inputs["input_ids"][0]),
        temperature=temperature,
        top_p=top_p,
    )
    # Decode and remove the prompt from the response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = full_response[len(prompt):].strip()  # Strip the prompt part
    return response

# Example
#prompt = "Give me the location data that has most in common with the following address: Hyllie Kyrkoväg 47, Limhamn?"
#response = generate_response(prompt, model, tokenizer)
#print(response)

In [6]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("json", data_files="data10.jsonl", split="train")

# Limit the dataset to the first 100 rows
limited_dataset = dataset.select(range(min(100, len(dataset))))

# Extract "Assistant" responses
assistant_responses = [row["text"].split("### Assistant:")[1].strip() for row in limited_dataset]

# Extract "Human" prompts
human_prompts = [row["text"].split("### Human:")[1].split("### Assistant:")[0].strip() for row in limited_dataset]


Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
# Generate and compare responses
results = []
for i, prompt in enumerate(human_prompts):
    model_response = generate_response(prompt, model, tokenizer)
    expected_response = assistant_responses[i]
    results.append({"Prompt": prompt, "Model Response": model_response, "Expected Response": expected_response})

# Optional: Print or save the results for analysis
import pandas as pd

results_df = pd.DataFrame(results)
results_df.to_csv("model_evaluation.csv", index=False)


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity(responses):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(responses)
    similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    return similarities

# Example for one comparison
similarities = [
    compute_similarity([row["Model Response"], row["Expected Response"]])[0]
    for _, row in results_df.iterrows()
]

# Add similarities to the DataFrame
results_df["Similarity"] = similarities

# Compute the total average similarity
total_average_similarity = sum(similarities) / len(similarities) if similarities else 0

# Add the total average similarity as a new column (if desired, for documentation purposes)
results_df["Total Average Similarity"] = total_average_similarity

# Save the results to a CSV file
results_df.to_csv("model_evaluation.csv", index=False)

print(f"Total Average Similarity: {total_average_similarity}")


Total Average Similarity: 0.15820730278708858


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_score, recall_score

def compute_precision_recall(model_response, expected_response):
    vectorizer = CountVectorizer(binary=True)
    response_matrix = vectorizer.fit_transform([model_response, expected_response])
    response_array = response_matrix.toarray()
    retrieved_words = set(vectorizer.get_feature_names_out(response_array[0] > 0))
    relevant_words = set(vectorizer.get_feature_names_out(response_array[1] > 0))
    
    intersection = retrieved_words & relevant_words
    precision = len(intersection) / len(retrieved_words) if retrieved_words else 0
    recall = len(intersection) / len(relevant_words) if relevant_words else 0
    
    return precision, recall

results_df["Precision"] = results_df.apply(lambda row: compute_precision_recall(row["Model Response"], row["Expected Response"])[0], axis=1)
results_df["Recall"] = results_df.apply(lambda row: compute_precision_recall(row["Model Response"], row["Expected Response"])[1], axis=1)


In [10]:
!pip install sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 1.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [11]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # Load a pre-trained model

def compute_faithfulness(model_response, expected_response):
    response_embedding = model.encode(model_response, convert_to_tensor=True)
    expected_embedding = model.encode(expected_response, convert_to_tensor=True)
    similarity = util.cos_sim(response_embedding, expected_embedding).item()
    return similarity

results_df["Faithfulness"] = results_df.apply(lambda row: compute_faithfulness(row["Model Response"], row["Expected Response"]), axis=1)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
def compute_relevancy(question, model_response):
    question_embedding = model.encode(question, convert_to_tensor=True)
    response_embedding = model.encode(model_response, convert_to_tensor=True)
    similarity = util.cos_sim(question_embedding, response_embedding).item()
    return similarity

results_df["Relevancy"] = results_df.apply(lambda row: compute_relevancy(row["Prompt"], row["Model Response"]), axis=1)


In [14]:
# Save to CSV
results_df.to_csv("model_evaluation_with_metrics.csv", index=False)

In [15]:
# Compute averages for all metrics
average_precision = results_df["Precision"].mean()
average_recall = results_df["Recall"].mean()
average_faithfulness = results_df["Faithfulness"].mean()
average_relevancy = results_df["Relevancy"].mean()

# Add these averages as a new row or save them separately
average_metrics = {
    "Metric": ["Precision", "Recall", "Faithfulness", "Relevancy"],
    "Average Value": [average_precision, average_recall, average_faithfulness, average_relevancy]
}

import pandas as pd
average_metrics_df = pd.DataFrame(average_metrics)

# Save averages to a separate file or append to the same file
average_metrics_df.to_csv("average_metrics.csv", index=False)

print("Average Metrics:")
print(average_metrics_df)

Average Metrics:
         Metric  Average Value
0     Precision       1.000000
1        Recall       1.000000
2  Faithfulness       0.461719
3     Relevancy       0.522230


In [19]:
# Input values
precision_values = [1, 1, 1, 1, 1]
recall_values = [1, 1, 1, 1, 1]
faithfulness_values = [0.8333, 0.5, 1, 1, 1]
relevancy_values = [0.975, 0.9611, 0.956893, 0.959725, 0.959725]

# Calculate averages
average_precision = sum(precision_values) / len(precision_values)
average_recall = sum(recall_values) / len(recall_values)
average_faithfulness = sum(faithfulness_values) / len(faithfulness_values)
average_relevancy = sum(relevancy_values) / len(relevancy_values)

# Create a dictionary for results
averages = {
    "Metric": ["Precision", "Recall", "Faithfulness", "Answer Relevancy"],
    "Average Value": [average_precision, average_recall, average_faithfulness, average_relevancy],
}

# Convert to DataFrame for display
import pandas as pd
averages_df = pd.DataFrame(averages)

# Print the results
print(averages_df)

             Metric  Average Value
0         Precision       1.000000
1            Recall       1.000000
2      Faithfulness       0.866660
3  Answer Relevancy       0.962489


In [18]:
# Input values
precision_values = [1, 1, 1, 1, 1]
recall_values = [1, 1, 1, 1, 1]
faithfulness_values = [1, 0.75, 1, 1, 1]
relevancy_values = [0.972, 0.932, 0.973, 0.972, 0.941]

# Calculate averages
average_precision = sum(precision_values) / len(precision_values)
average_recall = sum(recall_values) / len(recall_values)
average_faithfulness = sum(faithfulness_values) / len(faithfulness_values)
average_relevancy = sum(relevancy_values) / len(relevancy_values)

# Create a dictionary for results
averages = {
    "Metric": ["Precision", "Recall", "Faithfulness", "Answer Relevancy"],
    "Average Value": [average_precision, average_recall, average_faithfulness, average_relevancy],
}

# Convert to DataFrame for display
import pandas as pd
averages_df = pd.DataFrame(averages)

# Print the results
print(averages_df)

             Metric  Average Value
0         Precision          1.000
1            Recall          1.000
2      Faithfulness          0.950
3  Answer Relevancy          0.958
